<a href="https://colab.research.google.com/github/rajanaids-hub/Reinforcement_Learning_Lab/blob/main/TD_Learning_(SARSA)_Exp8_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
import random

# =====================================================================
# 🎓 STUDENT EXPERIMENT SETTINGS 🎓
# Change these numbers and observe how the agent's learning changes!
# =====================================================================
GRID_SIZE = 4           # Grid size (e.g., 4 makes a 4x4 grid)
EPISODES = 500          # How many "lives" or attempts the agent gets to learn
ALPHA = 0.1             # Learning Rate (0 to 1): 0 = learn nothing, 1 = overwrite completely
GAMMA = 0.9             # Discount Factor (0 to 1): How much does it care about future rewards?
EPSILON = 0.2           # Exploration Rate (0 to 1): 0.2 means 20% of the time it takes a random step
STEP_REWARD = -1.0      # Penalty for every step. -1 makes the agent want to finish quickly!
# =====================================================================

# 1. Define the Environment (The World the Agent lives in)
class GridworldEnv:
    def __init__(self, size=GRID_SIZE):
        self.size = size
        self.nS = size * size   # Total number of states (cells)
        self.nA = 4             # 4 Possible Actions: 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT

        # The goals are state 0 (top-left) and state nS-1 (bottom-right).
        # The agent can start anywhere else.
        self.start_states = list(range(1, self.nS - 1))

    def reset(self):
        """Put the agent in a random starting position at the beginning of an episode."""
        self.state = random.choice(self.start_states)
        return self.state

    def step(self, action):
        """The agent takes a step. The environment returns the new state and reward."""
        row, col = divmod(self.state, self.size)

        # Calculate new position based on action
        if action == 0: row = max(row - 1, 0)               # UP (Don't go off top edge)
        elif action == 1: col = min(col + 1, self.size - 1) # RIGHT (Don't go off right edge)
        elif action == 2: row = min(row + 1, self.size - 1) # DOWN (Don't go off bottom edge)
        elif action == 3: col = max(col - 1, 0)             # LEFT (Don't go off left edge)

        self.state = (row * self.size) + col

        # Did we hit Goal 1 (state 0) or Goal 2 (state nS-1)?
        done = (self.state == 0 or self.state == self.nS - 1)

        # If we reach the goal, the game ends (0 reward). Otherwise, penalty for taking a step.
        reward = 0.0 if done else STEP_REWARD

        return self.state, reward, done

# 2. The TD Learning Agent (The "Brain")
class SarsaAgent:
    def __init__(self, nS, nA, alpha, gamma, epsilon):
        self.nA = nA
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

        # The Q-Table is the agent's memory. It stores a "score" for every action in every state.
        # It starts at exactly zero, meaning the agent knows nothing!
        self.q_table = np.zeros((nS, nA))

    def choose_action(self, state):
        """Epsilon-Greedy Strategy: Balances exploring new paths vs exploiting known good paths."""
        if random.uniform(0, 1) < self.epsilon:
            # EXPLORE: Pick a completely random action
            return random.randint(0, self.nA - 1)
        else:
            # EXPLOIT: Look at the Q-table and pick the action with the highest score
            max_q = np.max(self.q_table[state])
            best_actions = np.where(self.q_table[state] == max_q)[0]
            # If multiple actions have the same top score, pick one randomly
            return random.choice(best_actions)

    def learn(self, state, action, reward, next_state, next_action):
        """
        The core of SARSA (State-Action-Reward-State-Action).
        This updates the Q-table using the Temporal Difference (TD) formula.
        """
        # 1. What do we currently think this state+action is worth?
        current_prediction = self.q_table[state, action]

        # 2. What is our prediction for the NEXT state+action?
        next_prediction = self.q_table[next_state, next_action]

        # 3. Calculate the TD Target: What actually happened + our guess for the future
        td_target = reward + (self.gamma * next_prediction)

        # 4. Calculate the TD Error: How wrong was our original prediction?
        td_error = td_target - current_prediction

        # 5. Update our Q-table using the learning rate (alpha) to move closer to the target
        self.q_table[state, action] = current_prediction + (self.alpha * td_error)

# =====================================================================
# 📉 VISUALIZATION HELPER FUNCTIONS
# =====================================================================
def extract_v_and_policy(q_table, nS, nA):
    """Converts the Q-table into a map of the best values and actions for plotting."""
    V = np.max(q_table, axis=1)
    policy = np.zeros((nS, nA))
    for s in range(nS):
        best_a = np.argmax(q_table[s])
        policy[s][best_a] = 1.0
    return V, policy

def animate_test_run(test_path, V, policy, size):
    """Creates a step-by-step animation of the trained agent walking to the goal."""
    fig, ax = plt.subplots(figsize=(6, 6))
    arrows = {0: '↑', 1: '→', 2: '↓', 3: '←'}
    start_state = size * (size - 1) # Bottom-Left corner

    def update(frame):
        ax.clear()
        ax.matshow(V.reshape(size, size), cmap='Blues') # Draw background colors
        agent_state = test_path[frame]

        for s in range(len(V)):
            row, col = divmod(s, size)

            # Draw standard grid text/arrows
            if s != 0 and s != len(V) - 1:
                best_act = np.argmax(policy[s])
                if s == start_state:
                    ax.text(col, row - 0.3, "START", va='center', ha='center', color='purple', fontsize=10, weight='bold')
                else:
                    ax.text(col, row - 0.2, f"{V[s]:.1f}", va='center', ha='center', color='black')
                ax.text(col, row + 0.2, arrows[best_act], va='center', ha='center', color='red', fontsize=16, weight='bold')
            else:
                # Highlight the two goals
                goal_label = "GOAL 1" if s == 0 else "GOAL 2"
                ax.text(col, row, f"{goal_label}\n🏁", va='center', ha='center', color='green', fontsize=11, weight='bold')

            # Draw the robot at its current position
            if s == agent_state:
                circle = plt.Circle((col, row), 0.3, color='red', alpha=0.5)
                ax.add_patch(circle)
                ax.text(col, row, "🤖", va='center', ha='center', fontsize=20)

        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"Trained Agent Path: Step {frame + 1}/{len(test_path)}", pad=20)

    # Compile the animation for display in Jupyter/Colab
    anim = animation.FuncAnimation(fig, update, frames=len(test_path), interval=500, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

# =====================================================================
# 🚀 MAIN EXECUTION LOOP (Training & Testing)
# =====================================================================
if __name__ == "__main__":
    env = GridworldEnv(size=GRID_SIZE)
    agent = SarsaAgent(nS=env.nS, nA=env.nA, alpha=ALPHA, gamma=GAMMA, epsilon=EPSILON)

    print(f"🧠 Training TD Agent (SARSA) for {EPISODES} episodes...")

    # --- TRAINING PHASE ---
    for episode in range(1, EPISODES + 1):
        state = env.reset()
        action = agent.choose_action(state)

        done = False
        while not done:
            # 1. Take action, observe reward and next state
            next_state, reward, done = env.step(action)

            # 2. Choose next action (SARSA needs to know the actual next action to learn)
            next_action = agent.choose_action(next_state)

            # 3. Update the agent's brain (Q-table)
            agent.learn(state, action, reward, next_state, next_action)

            # 4. Move forward in time
            state = next_state
            action = next_action

        if episode % 100 == 0:
            print(f"✅ Completed Episode {episode}")

    print("\n🎉 Training complete! Running a test episode...")

    # --- TESTING PHASE ---
    # Put the agent in the START tile (bottom-left)
    start_state = env.size * (env.size - 1)
    env.state = start_state
    state = start_state

    # Force agent to use its pure knowledge (no random exploration during test)
    agent.epsilon = 0.0

    test_path = [state] # Record the path it takes
    done = False
    max_steps = env.nS * 2 # Safety net to prevent infinite loops if agent failed to learn
    steps = 0

    while not done and steps < max_steps:
        action = agent.choose_action(state)
        next_state, reward, done = env.step(action)
        test_path.append(next_state)
        state = next_state
        steps += 1

    V, policy = extract_v_and_policy(agent.q_table, env.nS, env.nA)

    print("🎬 Rendering animation of the agent walking to the goal...")
    animate_test_run(test_path, V, policy, GRID_SIZE)

🧠 Training TD Agent (SARSA) for 500 episodes...
✅ Completed Episode 100
✅ Completed Episode 200
✅ Completed Episode 300
✅ Completed Episode 400
✅ Completed Episode 500

🎉 Training complete! Running a test episode...
🎬 Rendering animation of the agent walking to the goal...


/tmp/ipython-input-565673039.py:150: UserWarning: Glyph 127937 (\N{CHEQUERED FLAG}) missing from font(s) DejaVu Sans.
  display(HTML(anim.to_jshtml()))
/tmp/ipython-input-565673039.py:150: UserWarning: Glyph 129302 (\N{ROBOT FACE}) missing from font(s) DejaVu Sans.
  display(HTML(anim.to_jshtml()))
